In [3]:
import fasttext
import pandas as pd
import numpy as np

# Import Data

In [4]:
a = pd.read_json(r'C:\Users\nicol\Documents\travail\tfe\Thesis-main\dataset.json',encoding='utf-8')
df = [a["description"], a['header'],a['labels'], a['title']]  #specifing which columns to take into consideration when we do the classfication
df = pd.concat(df, axis=1, keys=["description", "header", "labels","title"])

# Preprocess (description and general labels)

In [5]:
#Description preprocess using treetagger
with open('description_prepro_with_TreeTagger.txt', 'r', encoding="utf-8") as f:
    temp = f.read() 
desc_treetag = (temp.split('\n'))
del(desc_treetag[-1])
df['desc_treetag'] = desc_treetag

#create collumn with only general labels
labs = ['atelier', 'autre', 'brocante', 'cinema', 'concert', 'conference',
       'etudiant', 'exposition', 'fete', 'lecture', 'spectacle', 'sport',
       'theatre']
general_labs = []
for tup in df.labels:
    glabs = []
    for word in tup:            
        if word in labs:
            glabs.append(word)
        else :
            for w in word.split("-"):
                if w in labs:
                    glabs.append(w)
                    
    glabs = list(dict.fromkeys(glabs))
    general_labs.append(glabs)

df['general_labels'] = general_labs

# Transform into format readable by FastText

In [6]:
def labforfasttext(labs):
    new_labs = ""
    for lab in labs:
        new_labs = str(new_labs) + '__label__' + lab + " "
    return new_labs

# NLP Preprocessing
from gensim.utils import simple_preprocess

# Importing the dataset
dataset = df[["desc_treetag", "general_labels"]]

# NLP Preprocess
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
lab_fasttext = []
for labs in df['labels'][:]:
    lab_fasttext.append(labforfasttext(labs))
dataset['lab_fasttext'] = lab_fasttext


c:\users\nicol\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
<ipython-input-6-884e652566c5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['lab_fasttext'] = lab_fasttext


# Train model

In [7]:
# Saving the CSV file as a text file to train/test the classifier
import csv, random
msk = np.random.rand(len(df)) < 0.8

dataset[msk][["desc_treetag", "lab_fasttext"]].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

dataset[~msk][["desc_treetag", "lab_fasttext"]].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")



# Training the fastText classifier
model = fasttext.train_supervised('train.txt', wordNgrams = 2, lr=0.3, epoch=100, loss='ova')


# Save the trained model
#model.save_model('model.bin')

# Test Model on One Event

In [8]:
i=0
for repeat in range(10):
    i += 1 
    while msk[i]:
        i = i+1
    print(dataset.iloc[i, 0])
    print(dataset.iloc[i, 1])
    print(model.predict(dataset.iloc[i, 0], k=-1, threshold=0.5))

comment connaissance scientifique produire quel liberté quel contrainte quel maniérer contexte monde rechercher santé compétition burn out rentabilité perte sens sen manquer temps inégalité genre quel influencer exercer niveau monde recherche scientifique découvrir vie rechercher travers témoignage personnel varié bonobos cosmologie passer nouveau technologie comment rechercher carrière tout tracée parcours attendre tour proche réalité quotidien aussi processus alternatif tel science café souhaiter lieu rencontrer entre espace favoriser dialogue débattre chercher ensemble piste condition travail tout public partir an entrer libre réservation souhaiter programmer complet activité organiser cadre printemps science où quand pointculture partenaire université catholique louvain scienceinfuse
['conference']
((), array([], dtype=float64))
artiste artisan producteur local nombreux tout ambiance féérique noël cœur ottignies ce édition art hiver ottignies déménager lieu emblématique ville porte

# Test Model on test set

In [9]:
print(model.test('test.txt', k=-1, threshold=0.5))

(192, 0.8225806451612904, 0.36298932384341637)
